In [1]:
import pandas as pd
import oracledb

# CSV 불러오기
df = pd.read_csv("C:/Users/itbank/OneDrive/바탕 화면/csv/카드소비 업종코드.csv")

# 컬럼명 변경
df.rename(columns={
    '업종코드(UPJONG_CD)': 'upjong_code',
    '대분류(CLASS1)': 'upjong',
}, inplace=True)

# Oracle 연결
conn = oracledb.connect(
    user="moa",
    password="moa",
    dsn="localhost:1521/XE"
) 

cursor = conn.cursor()

# INSERT
sql = """
INSERT INTO industry_codes (upjong_code, upjong)
VALUES (:1, :2)
"""
data = df[['upjong_code', 'upjong']].values.tolist()

cursor.executemany(sql, data)

conn.commit()
cursor.close()
conn.close()


In [2]:
import pandas as pd
import oracledb
from datetime import datetime

df = pd.read_csv("C:/Users/itbank/OneDrive/바탕 화면/csv/블록별 성별연령대별 카드소비패턴.csv")

# 컬럼명 DB용으로 변경
df.rename(columns={
    '서울시민업종코드(UPJONG_CD)': 'upjong_code',
    '기준년월(YM)': 'yearmonth',
    '성별(GEDNER)': 'gender',
    '카드이용금액계(AMT_CORR)': 'card_total'
}, inplace=True)

# ym 컬럼은 'YYYYMM' 문자열 → datetime 변환 (월 첫째날 기준)
def ym_to_date(ym_str):
    return datetime.strptime(str(ym_str), '%Y%m')

df['yearmonth'] = df['yearmonth'].apply(ym_to_date)

# id 컬럼 없으면 인덱스 기반으로 생성 (1부터 시작)
if 'gender_stats_id' not in df.columns:
    df.insert(0, 'gender_stats_id', range(1, len(df) + 1))
    
conn = oracledb.connect(
    user="moa",
    password="moa",
    dsn="localhost:1521/XE"
)

cursor = conn.cursor()

# Insert 쿼리
sql = """
INSERT INTO gender_stats (
    gender_stats_id, upjong_code, yearmonth, gender, card_total
) VALUES (
    :1, :2, :3, :4, :5
)
"""

# 데이터 리스트 생성
data = [
    (
        row['gender_stats_id'],
        row['upjong_code'],
        row['yearmonth'],
        row['gender'],
        row['card_total']
    )
    for _, row in df.iterrows()
]

# 데이터 삽입 실행
cursor.executemany(sql, data)

conn.commit()
cursor.close()
conn.close()

In [3]:
import pandas as pd
import oracledb
from datetime import datetime

df = pd.read_csv("C:/Users/itbank/OneDrive/바탕 화면/csv/블록별 성별연령대별 카드소비패턴.csv")

# 컬럼명 DB용으로 변경
df.rename(columns={
    '서울시민업종코드(UPJONG_CD)': 'upjong_code',
    '기준년월(YM)': 'yearmonth',
    '연령대별(AGE)': 'age_group',
    '카드이용금액계(AMT_CORR)': 'card_total'
}, inplace=True)

# ym 컬럼은 'YYYYMM' 문자열 → datetime 변환 (월 첫째날 기준)
def ym_to_date(ym_str):
    return datetime.strptime(str(ym_str), '%Y%m')

df['yearmonth'] = df['yearmonth'].apply(ym_to_date)

# id 컬럼 없으면 인덱스 기반으로 생성 (1부터 시작)
if 'age_stats_id' not in df.columns:
    df.insert(0, 'age_stats_id', range(1, len(df) + 1))
    
conn = oracledb.connect(
    user="moa",
    password="moa",
    dsn="localhost:1521/XE"
)

cursor = conn.cursor()

# Insert 쿼리
sql = """
INSERT INTO age_stats (
    age_stats_id, upjong_code, yearmonth, age_group, card_total
) VALUES (
    :1, :2, :3, :4, :5
)
"""

# 데이터 리스트 생성
data = [
    (
        row['age_stats_id'],
        row['upjong_code'],
        row['yearmonth'],
        row['age_group'],
        row['card_total']
    )
    for _, row in df.iterrows()
]

# 데이터 삽입 실행
cursor.executemany(sql, data)

conn.commit()
cursor.close()
conn.close()

In [4]:
import pandas as pd
import oracledb
from datetime import datetime

# CSV 읽기
df = pd.read_csv("C:/Users/itbank/OneDrive/바탕 화면/csv/블록별 시간대별소액결제건수 카드소비패턴.csv")

# 컬럼명 변경 (DB 컬럼명에 맞게)
df.rename(columns={
    '서울시민업종코드(UPJONG_CD)': 'upjong_code',
    '기준년월(YM)': 'yearmonth',
    '시간대구간(TIME)': 'time_slot',
    '카드이용금액계(AMT_CORR)': 'card_total',
}, inplace=True)

# 'ym' 컬럼: 'YYYYMM' → datetime(월 첫날 기준) 변환
def ym_to_date(ym_str):
    return datetime.strptime(str(ym_str), '%Y%m')

df['yearmonth'] = df['yearmonth'].apply(ym_to_date)

# id 컬럼 없으면 생성 (1부터 시작)
if 'payments_stats_id' not in df.columns:
    df.insert(0, 'payments_stats_id', range(1, len(df) + 1))

# Oracle DB 연결 정보 수정 필요
conn = oracledb.connect(
    user="moa",
    password="moa",
    dsn="localhost:1521/XE"
)

cursor = conn.cursor()

# Insert SQL (micro_payment_stats 테이블 컬럼에 맞게)
sql = """
INSERT INTO micro_payment_stats (
    payments_stats_id, upjong_code, yearmonth, time_slot, card_total
) VALUES (
    :1, :2, :3, :4, :5
)
"""

# 데이터 튜플 리스트 생성
data = [
    (
        row['payments_stats_id'],
        row['upjong_code'],
        row['yearmonth'],
        row['time_slot'],
        row['card_total']
    )
    for _, row in df.iterrows()
]

cursor.executemany(sql, data)

conn.commit()
cursor.close()
conn.close()


In [6]:
import pandas as pd
import oracledb
from datetime import datetime

# CSV 읽기 (처음부터 tot_reg_cd 문자열로 읽어도 되고, 읽은 후 변환해도 됨)
df = pd.read_csv("C:/Users/itbank/OneDrive/바탕 화면/csv/집계구별 일별소비지역별 카드소비패턴.csv")

# 컬럼명 변경 (DB 컬럼명에 맞게)
df.rename(columns={
    '업종코드(UPJONG_CD)': 'upjong_code',
    '가맹점주소광역시도(SIDO)': 'sido',
    '가맹점주소시군구(SGG)': 'sigungu',
    '업종대분류(UPJONG_CLASS1)': 'upjong',
    '기준일자(YMD)': 'yearmonthdate',
    '고객주소집계구별(TOT_REG_CD)': 'total_address_code',
    '카드이용금액계(AMT_CORR)': 'card_total',
}, inplace=True)

# 날짜 변환
def ymd_to_date(ymd_str):
    return datetime.strptime(str(ymd_str), '%Y%m%d')

df['yearmonthdate'] = df['yearmonthdate'].apply(ymd_to_date)

# id 컬럼 생성 (없으면)
if 'region_stats_id' not in df.columns:
    df.insert(0, 'region_stats_id', range(1, len(df) + 1))

# 문자열 컬럼들 (DB에서 VARCHAR2 타입 컬럼)
str_cols = ['upjong_code', 'sido', 'sigungu', 'upjong', 'total_address_code']

# 문자열 컬럼들을 문자열(str)로 강제 변환 (NaN/None은 빈 문자열로)
def to_str_safe(val):
    if val is None or (isinstance(val, float) and pd.isna(val)):
        return ''
    if isinstance(val, float):
        # float인 경우 정수로 바꿔서 문자열로 변환
        return str(int(val))
    return str(val)

# 문자열 컬럼들을 문자열(str)로 강제 변환
for col in str_cols:
    df[col] = df[col].apply(to_str_safe)

# DB 연결
conn = oracledb.connect(
    user="moa",
    password="moa",
    dsn="localhost:1521/XE"
)

cursor = conn.cursor()

# Insert SQL
sql = """
INSERT INTO region_stats (
    region_stats_id, upjong_code, sido, sigungu, upjong, yearmonthdate, total_address_code, card_total
) VALUES (
    :1, :2, :3, :4, :5, :6, :7, :8
)
"""

# 데이터 준비
data = [
    (
        row['region_stats_id'],
        row['upjong_code'],
        row['sido'],
        row['sigungu'],
        row['upjong'],
        row['yearmonthdate'],
        row['total_address_code'],
        row['card_total']
    )
    for _, row in df.iterrows()
]

# 실행
cursor.executemany(sql, data)

conn.commit()
cursor.close()
conn.close()
